# Step 1 : Importing Essetial Libraries

In [1]:
#!python --version

In [2]:
#!pip freeze

In [3]:
#from sklearn.feature_extraction.text import CountVectorizer



In [27]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers

In [28]:
max_length = 128  # Maximum length of input sentence to the model.
batch_size =4
epochs = 1

# Labels in our dataset.
labels = ["contradiction", "entailment", "neutral"]

In [29]:
# There are more than 550k samples in total; we will use 100k for this example.
train_df = pd.read_csv("/kaggle/input/datasetq1/SNLI_Corpus/snli_1.0_train.csv", nrows=100000)
valid_df = pd.read_csv("/kaggle/input/datasetq1/SNLI_Corpus/snli_1.0_dev.csv")
test_df = pd.read_csv("/kaggle/input/datasetq1/SNLI_Corpus/snli_1.0_test.csv")

# Shape of the data
print(f"Total train samples : {train_df.shape[0]}")
print(f"Total validation samples: {valid_df.shape[0]}")
print(f"Total test samples: {valid_df.shape[0]}")

Total train samples : 100000
Total validation samples: 10000
Total test samples: 10000


In [30]:
print(f"Sentence1: {train_df.loc[1, 'sentence1']}")
print(f"Sentence2: {train_df.loc[1, 'sentence2']}")
print(f"Similarity: {train_df.loc[1, 'similarity']}")

Sentence1: A person on a horse jumps over a broken down airplane.
Sentence2: A person is at a diner, ordering an omelette.
Similarity: contradiction


In [31]:
# We have some NaN entries in our train data, we will simply drop them.
print("Number of missing values")
print(train_df.isnull().sum())
train_df.dropna(axis=0, inplace=True)

Number of missing values
similarity    0
sentence1     0
sentence2     3
dtype: int64


In [32]:
print("Train Target Distribution")
print(train_df.similarity.value_counts())

Train Target Distribution
entailment       33384
contradiction    33310
neutral          33193
-                  110
Name: similarity, dtype: int64


In [33]:
print("Validation Target Distribution")
print(valid_df.similarity.value_counts())

Validation Target Distribution
entailment       3329
contradiction    3278
neutral          3235
-                 158
Name: similarity, dtype: int64


In [34]:
train_df = (
    train_df[train_df.similarity != "-"]
    .sample(frac=1.0, random_state=42)
    .reset_index(drop=True)
)
valid_df = (
    valid_df[valid_df.similarity != "-"]
    .sample(frac=1.0, random_state=42)
    .reset_index(drop=True)
)

In [35]:
train_df["label"] = train_df["similarity"].apply(
    lambda x: 0 if x == "contradiction" else 1 if x == "entailment" else 2
)
y_train = tf.keras.utils.to_categorical(train_df.label, num_classes=3)

valid_df["label"] = valid_df["similarity"].apply(
    lambda x: 0 if x == "contradiction" else 1 if x == "entailment" else 2
)
y_val = tf.keras.utils.to_categorical(valid_df.label, num_classes=3)

test_df["label"] = test_df["similarity"].apply(
    lambda x: 0 if x == "contradiction" else 1 if x == "entailment" else 2
)
y_test = tf.keras.utils.to_categorical(test_df.label, num_classes=3)

In [36]:

class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)


In [37]:
# Create the model under a distribution strategy scope.
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")

    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False

    bert_output = bert_model.bert(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    sequence_output = bert_output.last_hidden_state
    pooled_output = bert_output.pooler_output

    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, return_sequences=True)
    )(sequence_output)

    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.3)(concat)
    output = tf.keras.layers.Dense(3, activation="softmax")(dropout)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["acc"],
    )


print(f"Strategy: {strategy}")
model.summary()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Strategy: <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7beb9f283dd0>
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_masks (InputLayer)   [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['input_ids[0][0]',        

In [38]:
train_data = BertSemanticDataGenerator(
    train_df[["sentence1", "sentence2"]].values.astype("str"),
    y_train,
    batch_size=batch_size,
    shuffle=True,
)
valid_data = BertSemanticDataGenerator(
    valid_df[["sentence1", "sentence2"]].values.astype("str"),
    y_val,
    batch_size=batch_size,
    shuffle=False,
)

In [39]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2352: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


  258/24971 [..............................] - ETA: 24:41 - loss: 1.1124 - acc: 0.4215

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


  336/24971 [..............................] - ETA: 24:17 - loss: 1.1062 - acc: 0.4241

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


...

24950/24971 [============================>.] - ETA: 1s - loss: 0.6915 - acc: 0.7079

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


24971/24971 [==============================] - ETA: 0s - loss: 0.6915 - acc: 0.7080

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will alwa

24971/24971 [==============================] - 1673s 65ms/step - loss: 0.6915 - acc: 0.7080 - val_loss: 0.5399 - val_acc: 0.7878


In [28]:
# Unfreeze the bert_model.
bert_model.trainable = True
# Recompile the model to make the change effective.
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_masks (InputLayer)   [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_masks[0][0]',    

In [ ]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

In [41]:
test_df.head()

,similarity,sentence1,sentence2,label
0,neutral,This church choir sings to the masses as they ...,The church has cracks in the ceiling.,2
1,entailment,This church choir sings to the masses as they ...,The church is filled with song.,1
2,contradiction,This church choir sings to the masses as they ...,A choir singing at a baseball game.,0
3,neutral,"A woman with a green headscarf, blue shirt and...",The woman is young.,2
4,entailment,"A woman with a green headscarf, blue shirt and...",The woman is very happy.,1


In [42]:
y_test

array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)

In [43]:
new_df=test_df.head(3)
print(new_df)

      similarity                                          sentence1  \
0        neutral  This church choir sings to the masses as they ...   
1     entailment  This church choir sings to the masses as they ...   
2  contradiction  This church choir sings to the masses as they ...   

                               sentence2  label  
0  The church has cracks in the ceiling.      2  
1        The church is filled with song.      1  
2    A choir singing at a baseball game.      0  


In [51]:
new_df.head()

,similarity,sentence1,sentence2,label
0,neutral,This church choir sings to the masses as they ...,The church has cracks in the ceiling.,2
1,entailment,This church choir sings to the masses as they ...,The church is filled with song.,1
2,contradiction,This church choir sings to the masses as they ...,A choir singing at a baseball game.,0


In [47]:
y_test2=y_test[0:3]

In [48]:
y_test2

array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]], dtype=float32)

In [50]:
test_data2 = BertSemanticDataGenerator(
    new_df[["sentence1", "sentence2"]].values.astype("str"),
    y_test2,
    batch_size=batch_size,
    shuffle=False,
)
model.evaluate(test_data2, verbose=1)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:239: RuntimeWarning: divide by zero encountered in log10
  numdigits = int(np.log10(self.target)) + 1


OverflowError: cannot convert float infinity to integer

In [64]:
# Reduced dataset
new_df = test_df.head(3)
print(new_df)

# Corresponding labels
y_test2 = y_test[0:3]

# Adjust the batch size if necessary
adjusted_batch_size = min(batch_size, len(new_df))

# Create a new data generator with the adjusted batch size
test_data2 = BertSemanticDataGenerator(
    new_df[["sentence1", "sentence2"]].values.astype("str"),
    y_test2,
    batch_size=adjusted_batch_size,
    shuffle=False,
)

# Evaluate the model with the new data generator
model.evaluate(test_data2, verbose=1)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


      similarity                                          sentence1  \
0        neutral  This church choir sings to the masses as they ...   
1     entailment  This church choir sings to the masses as they ...   
2  contradiction  This church choir sings to the masses as they ...   

                               sentence2  label  
0  The church has cracks in the ceiling.      2  
1        The church is filled with song.      1  
2    A choir singing at a baseball game.      0  


/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2352: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


1/1 [==============================] - 0s 353ms/step - loss: 1.0127 - acc: 0.6667


[1.0126597881317139, 0.6666666865348816]

In [53]:
#test it on custom sentences

In [71]:
# Reduced dataset
new_df = test_df.head(3)
print(new_df)

# Corresponding labels
y_test2 = y_test[0:3]


import pickle

# Save new_df as a pickle file
with open('/kaggle/working/new_df.pkl', 'wb') as f:
    pickle.dump(new_df, f)

# Save y_test2 as a pickle file
with open('/kaggle/working/y_test2.pkl', 'wb') as f:
    pickle.dump(y_test2, f)

# Load new_df from the pickle file
with open('/kaggle/working/new_df.pkl', 'rb') as f:
    loaded_new_df = pickle.load(f)


# Modify the first row's sentence1 and sentence2 values
loaded_new_df.at[0, 'sentence1'] = "there are so many problems in the world we cant solve them."
loaded_new_df.at[0, 'sentence2'] = "today i feel very very happy for what happened."

# Display the modified dataframe
print(loaded_new_df)    

# Load y_test2 from the pickle file
with open('/kaggle/working/y_test2.pkl', 'rb') as f:
    loaded_y_test2 = pickle.load(f)

    
# Adjust the batch size if necessary
adjusted_batch_size = min(batch_size, len(loaded_new_df))

# Create a new data generator with the loaded data
test_data2 = BertSemanticDataGenerator(
    loaded_new_df[["sentence1", "sentence2"]].values.astype("str"),
    loaded_y_test2,
    batch_size=adjusted_batch_size,
    shuffle=False,
)


# Evaluate the model with the new data generator
model.evaluate(test_data2, verbose=1)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


      similarity                                          sentence1  \
0        neutral  This church choir sings to the masses as they ...   
1     entailment  This church choir sings to the masses as they ...   
2  contradiction  This church choir sings to the masses as they ...   

                               sentence2  label  
0  The church has cracks in the ceiling.      2  
1        The church is filled with song.      1  
2    A choir singing at a baseball game.      0  
      similarity                                          sentence1  \
0        neutral  there are so many problems in the world we can...   
1     entailment  This church choir sings to the masses as they ...   
2  contradiction  This church choir sings to the masses as they ...   

                                         sentence2  label  
0  today i feel very very happy for what happened.      2  
1                  The church is filled with song.      1  
2              A choir singing at a baseball game.

/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2352: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


1/1 [==============================] - 0s 338ms/step - loss: 0.3752 - acc: 0.6667


[0.3752380311489105, 0.6666666865348816]

In [76]:
print(adjusted_batch_size)

3


In [72]:
# Predict with the new data generator
predictions = model.predict(test_data2, verbose=1)

# Print the predictions
print(predictions)


1/1 [==============================] - 0s 322ms/step
[[0.55687267 0.01365298 0.42947432]
 [0.04960193 0.8360035  0.1143945 ]
 [0.90357316 0.00267221 0.0937546 ]]


In [68]:
# Save the model
model.save("/kaggle/working/trained_model.h5")


In [69]:
from keras.models import load_model

# Load the model
loaded_model = load_model("/kaggle/working/trained_model.h5")


In [70]:
# Predict with the loaded model
predictions_loaded_model = loaded_model.predict(test_data2, verbose=1)

# Print the predictions
print(predictions_loaded_model)


1/1 [==============================] - 4s 4s/step
[[0.16296065 0.02428008 0.8127593 ]
 [0.04960195 0.8360034  0.11439458]
 [0.90357316 0.00267221 0.09375466]]


In [40]:
test_data = BertSemanticDataGenerator(
    test_df[["sentence1", "sentence2"]].values.astype("str"),
    y_test,
    batch_size=batch_size,
    shuffle=False,
)
model.evaluate(test_data, verbose=1)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


 580/2500 [=====>........................] - ETA: 1:37 - loss: 0.5811 - acc: 0.7672

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


 685/2500 [=======>......................] - ETA: 1:32 - loss: 0.5733 - acc: 0.7697

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


2500/2500 [==============================] - 127s 50ms/step - loss: 0.5639 - acc: 0.7739


[0.5638877749443054, 0.7738999724388123]

In [73]:

def check_similarity(sentence1, sentence2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )

    proba = model.predict(test_data[0])[0]
    idx = np.argmax(proba)
    proba = f"{proba[idx]: .2f}%"
    pred = labels[idx]
    return pred, proba


In [74]:
# saving model in .h5 file for deployment
#model.save('/kaggle/working/saved_model_answerbot.h5')

In [75]:
sentence1 = "Two women are observing something together."
sentence2 = "Two women are standing with their eyes closed."
check_similarity(sentence1, sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


CancelledError: Graph execution error:

RecvAsync is cancelled.
	 [[{{node GroupCrossDeviceControlEdges_0/Identity_2/_905}}]] [Op:__inference_predict_function_346190]

In [ ]:
sentence1 = "A smiling costumed woman is holding an umbrella"
sentence2 = "A happy woman in a fairy costume holds an umbrella"
check_similarity(sentence1, sentence2)

In [ ]:
sentence1 = "A soccer game with multiple males playing"
sentence2 = "Some men are playing a sport"
check_similarity(sentence1, sentence2)

In [ ]:
# 1. Save the model
model.save('saved_model_pooja.h5')

# 2. Compress the file (optional)
import shutil
shutil.make_archive('/kaggle/working/saved_model_pooja', 'zip', '/kaggle/working', 'saved_model_pooja.h5')

# 3. Generate the download link and automatically trigger the download
import IPython

# Create HTML for the download link
file_path = '/kaggle/working/saved_model_pooja.zip'
file_name = 'saved_model_pooja.zip'

html = f'''
<html>
<body>
    <a id="download_link" href="{file_path}" download="{file_name}">Download</a>
    <script>
        document.getElementById('download_link').click();
    </script>
</body>
</html>
'''

# Display the HTML
IPython.display.display(IPython.display.HTML(html))
